# Music Taste Discovery

Importing confidential variables

In [3]:
from spotify_cred import client_id, client_secret, user_id

Importing important libraries to get data, clean it, and make it uasable for analysis

In [2]:
import requests                                                         # For making HTTP requests to the Spotify API

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials                     # To access authorised Spotify (public) data

In [ ]:
import pandas as pd                                                    # To analyse/visualize data in a tabular format

In [7]:
num_playlist = 22                                                      # Number of playlist you want to pull

output_type = "application/json"                                       # Output format from Spotify API
content_type = "application/json"

# Change auth_token before running code
auth_token = ""


query_headers = {"Accept" : output_type,                               # Headers for the requests.get() method
                 "Content-Type" : content_type,
                 "Authorization" : f'Bearer {auth_token}'}

In [ ]:
endpoint_url = f'https://api.spotify.com/v1/users/{user_id}/playlists' # Endpoint url of interest (user playlists)

query = f'{endpoint_url}?limit={num_playlist}'                         # Basic url query

response = requests.get(query,                                         # API call
               headers = query_headers)
response

In [ ]:
json_response = response.json()                                        # Creating dict using .json() method

In [ ]:
# Write the output response to file
with open('kc941 playlists.json', 'w', encoding='utf-8') as file:
    json.dump(json_response, file, ensure_ascii=False, indent=4)

In [ ]:
# Opening JSON file
f = open('kc941 playlists.json',)
  
# returns JSON object as 
# a dictionary
spotify_playlists = json.load(f)

# Closing file
f.close()

In [ ]:
playlist_simplified = {}


# iterating over number of playlist
# playlist name becomes key, playlist uri becomes value

for i in range(len(spotify_playlists['items'])):
    playlist_simplified[spotify_playlists['items'][i]['name']] = spotify_playlists['items'][i]['uri']

# converting playlist dictionary to DataFrame
playlist_df = pd.DataFrame.from_dict(playlist_simplified, orient = 'index', columns = ['URI'])


playlist_df['Playlist'] = playlist_df.index                          # Default playlist index is name, making a column from index
playlist_df.index = [x for x in range(len(playlist_df))]             # Creating new index



playlist_df

In [1]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

# name = "Alan Walker" #chosen artist
# result = sp.search(name) #search query
# result['tracks']['items'][0]['artists']

NameError: name 'SpotifyClientCredentials' is not defined